In [ ]:
## Imports
from atproto import Client, models
from atproto.exceptions import AtProtocolError
import csv
import pandas as pd
import json
import time

In [ ]:
# Autenticação na conta BlueSky
client = Client()
client.login("lcstuber.net", "bxax-ucco-xz2d-gseu")

# Nome de usuário
username = "gatinha01.bsky.social"


In [ ]:
data = client.get_timeline(limit=30)

feed = data.feed
next_page = data.cursor

In [ ]:
def get_all_followers(handle: str):
    try:
        profile = client.com.atproto.identity.resolve_handle({'handle': handle})
        did = profile['did']

        followers = []
        cursor = None

        while True:
            response = client.app.bsky.graph.get_followers({
                'actor': did,
                'limit': 100,
                'cursor': cursor
            })

            followers_batch = response['followers']
            followers.extend(followers_batch)

            # Opcional: log de progresso
            print(f"Coletados {len(followers)} seguidores até agora...")

            # Checa se há um cursor para continuar
            cursor = response.cursor
            if not cursor:
                break

        return followers

    except AtProtocolError as e:
        print(f"Erro na API: {e}")
        return []

In [ ]:
handle = 'duolingobrasil.com.br'
followers = get_all_followers(handle)
print(f'Total de seguidores: {len(followers)}')

In [ ]:
def get_all_posts_of_user(did: str):
    all_posts = []
    cursor = None

    while True:
        try:
            response = client.app.bsky.feed.get_author_feed({
                'actor': did,
                'limit': 100,
                'cursor': cursor
            })

            feed = response.feed

            if not feed:
                break

            for post in feed:
                post_data = {
                    'uri': post.post.uri,
                    'text': post.post.record.text,
                    'created_at': post.post.record.created_at
                }
                all_posts.append(post_data)

            cursor = response.cursor
            if not cursor:
                break

            time.sleep(0.2)

        except Exception as e:
            print(f"Erro ao buscar posts de {did}: {e}")
            break

    return all_posts

In [ ]:
def save_incrementally(data, filename='followers_posts.json'):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = []

    existing_data.append(data)

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)

    print(f"Dados salvos com sucesso.")

In [ ]:
for follower in followers:
    follower_did = follower['did']
    follower_handle = follower['handle']

    print(f"Buscando posts de @{follower_handle} ({follower_did})...")

    posts = get_all_posts_of_user(follower_did)

    user_data = {
        'posts': posts
    }

    save_incrementally(user_data)

    print(f"Coletados {len(posts)} posts de @{follower_handle}.\n")

    time.sleep(0.5)